# Lecture 2 - Functional Programming

## 📕 Today's Agenda
---
 * [Static methods and class attributes](#Static-methods-and-class-attributes)
 * [Object copy](#Object-copy)
 * [Iterators](#Iterators)
 * [Generators](#Generators)
 * [Lambda functions](#Lambda-functions)
 * [List comprehension](#List-comprehension)

## 🧪 Theory
---
### Static methods and class attributes
**Class attributes definition**

Static attributes are sharing the value with all instances. They are inherited by all child classes.
Static attributes can be accesses also as instance attribute only if a instance attribute with same name does not exist,
otherwise the instance attribute will be accessed.

In [1]:
class Connection:
    default_port = 9000 # define and assign value for static attribute

class Car:
    cars_created = 0 # define and assign value for static attribute

    def __init__(self):
        Car.cars_created += 1 # set value for instance attribute
        #self.cars_created = 888 # instance attributes have priority

class Volvo(Car):

    def __init__(self):
        super().__init__()
        self.x = 1

v = Volvo()
print(v.cars_created) # print instance attribute
print(dir(v))
v.cars_created = 4 # define and set instance attribute
print(v.cars_created)
print('Static value', Car.cars_created) # print class attribute
vv = Volvo() # create new Car
print(v.cars_created) # print instance attribute
print(Car.cars_created) # print class attribute

1
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'cars_created', 'x']
4
Static value 1
4
2


**Static methods**

A static method is a method defined in a class but with no access to instance attributes or methods.
Static methods can be used to compute data based on static attributes.
When a class contains only static methods it is considered as namespace.

In [2]:
class Math:
    pi = 3.14
    # define static method using @staticmethod decorator
    @staticmethod
    def pow(a, b): # self argument is no longer needed
        return a ** b

    @staticmethod
    def mean(seq):
        return sum(seq) / len(seq)

    @staticmethod
    def circle_area(r):
        return 2 * Math.pi * r # use static attributes

print(Math.pow(2, 5)) # call a static method
print(Math.circle_area(3))

32
18.84


### Object copy

The problem: you can not copy an object just by using equal sign.

In [3]:
volvo_v60 = Volvo()
volvo_v61 = volvo_v60
print('Are the same object?', volvo_v60 is volvo_v61)
print('ID volvo_v60', id(volvo_v60))
print('ID volvo_v61', id(volvo_v61))

Are the same object? True
ID volvo_v60 1991324626176
ID volvo_v61 1991324626176


The solution 😊

In [4]:
from copy import copy
volvo_v70 = copy(volvo_v60)
print('Are the same object?', volvo_v60 is volvo_v70)
print('ID volvo_v60', id(volvo_v60))
print('ID volvo_v70', id(volvo_v70))

Are the same object? False
ID volvo_v60 1991324626176
ID volvo_v70 1991324626560


Compounded objects issue

In [5]:
class User:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

class Notebook:
    def __init__(self, title: str, owner: User):
        self.title = title
        self.owner = owner

notebook = Notebook('Tests', User('Mihai', 'Dinu'))
notebook_copy = copy(notebook)

print('Original ID', id(notebook))
print('Copy ID', id(notebook_copy))
print("-" * 32)
print('Original User ID', id(notebook.owner))
print('Copy User ID', id(notebook_copy.owner))
print('Are them the same object?', 'Yes' if notebook_copy.owner is notebook.owner else 'No')

Original ID 1991324529472
Copy ID 1991324873008
--------------------------------
Original User ID 1991324530048
Copy User ID 1991324530048
Are them the same object? Yes


Solution? Deep Copy

In [6]:
from copy import deepcopy
notebook_copy2 = deepcopy(notebook)
print('Original ID', id(notebook))
print('Copy ID', id(notebook_copy2))
print("-" * 32)
print('Original User ID', id(notebook.owner))
print('Copy User ID', id(notebook_copy2.owner))
print('Are them the same object?', 'Yes' if notebook_copy2.owner is notebook.owner else 'No')

Original ID 1991324529472
Copy ID 1991324780192
--------------------------------
Original User ID 1991324530048
Copy User ID 1991324780000
Are them the same object? No


In order to change the behaviour for copy and deepcopy functions for a custom class, the __copy__ and __deepcopy__ methods
will be overwritten. This is used when you want to copy just some of attributes. Both magic methods must return a new object.

### Iterators

An **iterable** object is an object that implements \_\_iter\_\_, which is expected to return an **iterator** object.

An iterator is an object that implements next, which is expected to return the next element of the iterable object that
returned it, and raise a *StopIteration* exception when no more elements are available.

In [7]:
# basic iterable class

import random

class DummyIter:
    pass

class TenRandInt:
    """This class will return an iterator that returns 10 random integers."""
    def __init__(self):
        self.count = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.count < 10:
            self.count += 1
            return random.randint(0, 10000)
        else:
            raise StopIteration # if already generated 10 ints, raise StopIteration in order to break the loop

# normal class instance
rand_ints = TenRandInt()

# get iterator object for rand_ints object
rand_ints_iter = iter(rand_ints)
print('Iterable object:', type(rand_ints), id(rand_ints))
print('Iterator object:', type(rand_ints_iter), id(rand_ints))

# check if an object can be iterated
try:
    print(iter(DummyIter()))
except TypeError as err:
    print('[ERROR] ', err)

print('DummyIter is iterable? ', hasattr(DummyIter(), '__iter__'))
print('TenRandInt is iterable? ', hasattr(TenRandInt(), '__iter__'))

print('DummyIter is an iterator? ', hasattr(DummyIter(), '__next__'))
print('TenRandInt is an iterator? ', hasattr(TenRandInt(), '__next__'))

# obtain next value from an iterator
print('\nValue returned by iterator:')
print(next(rand_ints))

# iterate with for through iterator elements
print('\nAll iterator values:')
for i in rand_ints:
    print(i)

for i in rand_ints:
    print(i)

Iterable object: <class '__main__.TenRandInt'> 1991324874640
Iterator object: <class '__main__.TenRandInt'> 1991324874640
[ERROR]  'DummyIter' object is not iterable
DummyIter is iterable?  False
TenRandInt is iterable?  True
DummyIter is an iterator?  False
TenRandInt is an iterator?  True

Value returned by iterator:
3632

All iterator values:
6106
4522
9919
7469
9360
9939
5056
5537
3127


### Generators

Python Generators are a special kind of functions that return a lazy iterator, so they are iterable, but unlike
lists they don't have to hold all the values in memory due to their awesome ability to retain internal state between calls. They
save memory, but are slower than other iterables, so there is a tradeoff.

**Usages:**
 - generate number sequences in which numbers are based on previous numbers (like Fibonacci)
 - when reading file system tree
 - reading large files
 - reading stream data
 - generate infinite data sequences

**Definition:**

In [8]:
def fib():
    a = 0
    b = 1
    while True:
        a, b = b, a + b
        yield b

**Usage:**

In [9]:
fib_gen = fib()
print(type(fib_gen))
print(next(fib_gen))
print(next(fib_gen))
print(next(fib_gen))
print(next(fib_gen))

for i in range(100):
    print(next(fib_gen))

<class 'generator'>
1
2
3
5
8
13
21
34
55
89
144
233
377
610
987
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393
196418
317811
514229
832040
1346269
2178309
3524578
5702887
9227465
14930352
24157817
39088169
63245986
102334155
165580141
267914296
433494437
701408733
1134903170
1836311903
2971215073
4807526976
7778742049
12586269025
20365011074
32951280099
53316291173
86267571272
139583862445
225851433717
365435296162
591286729879
956722026041
1548008755920
2504730781961
4052739537881
6557470319842
10610209857723
17167680177565
27777890035288
44945570212853
72723460248141
117669030460994
190392490709135
308061521170129
498454011879264
806515533049393
1304969544928657
2111485077978050
3416454622906707
5527939700884757
8944394323791464
14472334024676221
23416728348467685
37889062373143906
61305790721611591
99194853094755497
160500643816367088
259695496911122585
420196140727489673
679891637638612258
1100087778366101931
1779979416004714189
2880067194370816120
4660046610375530309
75

### Lambda functions

Known as in line functions or anonymous functions. They are used to make fast inline calculus or as argument to a function
that needs a function to do something. You will see lambda functions mostly used as argument for map, filter, sort etc.

**Definition:**

In [10]:
lambda x: x

<function __main__.<lambda>(x)>

- *lambda* keyword
- argument list
- colon sign :
- expresion to return

So in example above the lambda function will take an argument and return it as it is.

**Lambda without arguments:**

In [11]:
lambda: "Hello World" # it will return "Hello World"

print(lambda: "Hello World")
print((lambda: "Hello World")())

<function <lambda> at 0x000001CFA437AA60>
Hello World


**Lambda assignment:**

In [12]:
square = lambda x: x ** 2
print(square(23))

529


**Lambda usage as argument:**

In [13]:
list2 = [1, 2, 3]
print(list(map(lambda x: x * 2, list2)))

[2, 4, 6]


**Function equivalent for lambda:**

In [14]:
def double(x):
    return x * 2

print(list(map(double, list2)))

[2, 4, 6]


### List comprehension

List comprehension is a way of generating new lists in Python starting from an existing list or an expresion.
List comprehension works kind same as *map* function, but it does not change input list, it creates a new one.

**General form:**

In [15]:
l1 = [2, 4, 6, 8]
l2 = [x for x in l1] # it creates a copy of l1

**Syntax explanation:**
- open bracket - [
- what to append to the new list - *x*
- iteration - *for x in l1*
- closing bracket - ]

In [16]:
l3 = [x * 2 for x in l1] # it creates a copy of l1 but each element if multiplied by 2
print(l3)

[4, 8, 12, 16]


**Explicit version:**

In [17]:
l4 = []
for x in l1:
    l4.append(x * 2)
print(l4)

[4, 8, 12, 16]


**Used for filtering with *if*:**

In [18]:
l5 = [x for x in l4 if not x % 3]
print(l5)

[12]


**Explicit form for filtering with *if*:**

In [19]:
l6 = []
for x in l4:
    if not x % 3:
        l6.append(x)

print(l6)

[12]


**Double *for* compression:**

In [20]:
matrix = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
]

flat_matrix = [x for line in matrix for x in line]
print(flat_matrix)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


**Explicit double *for*:**

In [21]:
flat_matrix2 = []
for line in matrix:
    for x in line:
        flat_matrix2.append(x)
print(flat_matrix2)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


## 👩‍💻 Practice
---
1. Write a class that keeps track of its objects in a static list.
2. Write a class which overwrites the \_\_copy\_\_ method so you can check if an object has copies.
```python
shop = Shop()
shop_copy = copy(shop)
shop.has_copies() -> bool
```
3. Write a class that represents english alphabet and make it iterable so you can do *for* over this class and get each alphabet letter.
4. Write a lambda function that test if a number if even, will return a bool. Use this lambda as argument for
*filter* function to extract a list of even numbers from a given list.
5. Write a list comprehension to solve point 4.

## 🏠 Homework
---
1. Write a script that reads a [file](cnps.txt) containing CNP's using generators, so each CNP should be yielded.
 - check for CNP validity
 - calculate men's age mean
 - calculate women's age mean
 - print CNP for all under mean mens

NOTE: use list comprehension to filter CNPs.